In [ ]:
# default_exp model

In [ ]:
#export
from fastai2.vision.all import *
import torchaudio
import torchaudio.transforms

In [ ]:
#hide
from nbdev.showdoc import *

# fastai2-resnet-audio model

> ResNet-like 1D CNN model for audio

## Dependencies
- fastai2
- torchaudio

## ResNet-like 1D CNN

This code is inspired by https://www.kaggle.com/readilen/resnet-for-mnist-with-pytorch, https://towardsdatascience.com/understanding-and-visualizing-resnets-442284831be8 and 
https://github.com/fastai/fastai2/blob/master/nbs/11_vision.models.xresnet.ipynb

In [ ]:
#export

def conv1xk(in_channels, out_channels, kernel_size=3, stride=1):
    padding = kernel_size // 2
    return nn.Conv1d(in_channels, out_channels, kernel_size=kernel_size,
                    stride=stride, padding=padding, bias=False)

def init_cnn_1d(m):
    if getattr(m, 'bias', None) is not None: nn.init.constant_(m.bias, 0)
    if isinstance(m, (nn.Conv1d,nn.Linear)): nn.init.kaiming_normal_(m.weight)
    for l in m.children(): init_cnn_1d(l)
        
def  splitter(m):
    return L(m[0][:6], m[0][6:], m[1]).map(params)

# Residual block
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size=3, stride=1, downsample=None):
        super(ResidualBlock, self).__init__()
        self.conv1 = conv1xk(in_channels, out_channels, kernel_size, stride)
        self.bn1 = nn.BatchNorm1d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv1xk(out_channels, out_channels, kernel_size)
        self.bn2 = nn.BatchNorm1d(out_channels)
        self.downsample = downsample

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample:
            residual = self.downsample(residual)
        out += residual
        out = self.relu(out)
        return out
    
    
class ResNetAudio(nn.Sequential):
    def __init__(self, block, layers, in_channels=64, num_classes=10, kernel_size=3, stride=2, dropout=0.2):
        in_block = [] # input block
        residual_block = [] # residual blocks
        header_block = [] # linear head
        
        self.in_channels = in_channels
        self.block = block
        
        in_kernel_size = kernel_size * 2 + 1
        
        in_block.append(conv1xk(1, in_channels, in_kernel_size, stride))
        in_block.append(nn.BatchNorm1d(in_channels))
        in_block.append(nn.ReLU(inplace=True))
        in_block.append(nn.MaxPool1d(kernel_size, stride, kernel_size//3))
        
        residual_block = self.make_blocks(layers, in_channels, kernel_size, stride)
        
        header_block.append(nn.AdaptiveAvgPool1d(1))
        header_block.append(nn.Flatten())
        header_block.append(nn.Dropout(dropout))
        header_block.append(nn.Linear(in_channels*2**(len(layers)-1), num_classes))
        
        super().__init__(nn.Sequential(*in_block, *residual_block),nn.Sequential(*header_block))
        init_cnn_1d(self)

    def make_blocks(self, layers, in_channels, kernel_size, stride):
        return [self.make_layer(self.block, in_channels*2**i, l, kernel_size, stride) for i, l in enumerate(layers)]
        
    def make_layer(self, block, out_channels, blocks, kernel_size=3, stride=1):
        downsample = None
        if (stride != 1) or (self.in_channels != out_channels):
            downsample = nn.Sequential(
                conv1xk (self.in_channels, out_channels, kernel_size=kernel_size, stride=stride),
                nn.BatchNorm1d(out_channels))
        layers = []
        layers.append(block(self.in_channels, out_channels, kernel_size, stride, downsample))
        self.in_channels = out_channels
        for i in range(1, blocks):
            layers.append(block(out_channels, out_channels, kernel_size))
        return nn.Sequential(*layers)
        

## Fine-Tunable model

Call replace_head(learn.model, num_classes=<new_num_classes>) to adapt the model head to a new dataset (new number of classes). Now you can use learn.fine_tune(), learn.unfreeze() and learn.fit_one_cycle() to fine-tune the model. 

In [ ]:
#export

def replace_head(model, num_classes):
    model[-1][-1] = nn.Linear(512, num_classes)
    apply_init(model[1], nn.init.kaiming_normal_)

## Configurations

Configurations for **ResNet18 / ResNet34-like** architectures. Kernel size 34 and stride 4 seem to work quite 
well. But there is still room for experimentation and improvement.


In [ ]:
#export

# resnet 18
resnet1d18 = {
    "block": ResidualBlock,
    "layers": [2, 2, 2, 2],
    "in_channels": 64,
    "kernel_size": 15,
    "stride": 4,
    "num_classes": 10
}

# resnet 34
resnet1d34 = {
    "block": ResidualBlock,
    "layers": [3, 4, 6, 3],
    "in_channels": 64,
    "kernel_size": 15,
    "stride": 4,
    "num_classes": 10
}


## Tests

Test architecutre

In [ ]:
#hide
bs = 8
arch = resnet1d18
model = ResNetAudio(**arch)
inp = torch.randn(bs,1,22050)
out = model(inp)
assert len(out) == bs
assert len(out[0]) == arch['num_classes']

Test replace_head

In [ ]:
#hide
num_classes = 22
replace_head(model, num_classes)
assert getattr(model[-1][-1], 'out_features') == num_classes